## .top file parser functions

In [1]:
def read_rawdata(file_nr):
    folder = "../Parameter files (.top)/"
    filename = "NVE_par_"+str(file_nr)+".top"
    filepath = folder + filename

    with open(filepath, 'rb') as f:
        rawdata = f.read()
    
    # Cast byte to string data
    return str(rawdata)    

In [2]:
def get_param_value(data, key: str):
    index = data.find(key)
    
    # NB hardcoded read indexes, found by trial 
    OFFSET1 = 43
    OFFSET2 = -7
    
    value = float(data[index+OFFSET1+OFFSET2:index+OFFSET1])
    return value

## Key data to read from .top file

In [3]:
## Interesting key data to read

# Elevation data
ELEVS = ["ELEV"+str(i) for i in range(11)]

# Evaporation per month
EVAPOS = ["EPJAN", "EPFEB", "EPMAR", "EPAPR", "EPMAY", "EPJUN", "EPJUL", "EPAUG", "EPSEP", "EPOKT", "EPNOV", "EPDES"]

# Other params 
OTHER = ["RCORR", "SCORR", "PGRAD", "TPGRAD"]

interesting_keys = [ELEVS, OTHER, EVAPOS]
# Flatten lists inside list
interesting_keys = [val for sublist in interesting_keys for val in sublist]

In [4]:
def read_interesting_params(data, keys):
    params_dict = {}
    for key in keys: 
        params_dict[key] = get_param_value(data, key)
    return params_dict

In [5]:
file_nr = 1
data = read_rawdata(file_nr)
read_interesting_params(data, interesting_keys)

{'ELEV0': 670.0,
 'ELEV1': 894.0,
 'ELEV2': 23.0,
 'ELEV3': 109.0,
 'ELEV4': 196.0,
 'ELEV5': 261.0,
 'ELEV6': 304.0,
 'ELEV7': 335.0,
 'ELEV8': 361.0,
 'ELEV9': 391.0,
 'ELEV10': 534.0,
 'RCORR': 1.594,
 'SCORR': 1.768,
 'PGRAD': -0.5,
 'TPGRAD': -0.0,
 'EPJAN': 0.1,
 'EPFEB': 0.2,
 'EPMAR': 0.7,
 'EPAPR': 1.0,
 'EPMAY': 2.3,
 'EPJUN': 3.5,
 'EPJUL': 3.5,
 'EPAUG': 2.3,
 'EPSEP': 1.0,
 'EPOKT': 0.7,
 'EPNOV': 0.2,
 'EPDES': 0.1}

## Read Output, Simulated and Residuals data

In [19]:
import pandas as pd

file_nr = 1

def read_output_simulated_residual_data(file_nr):
    # Read raw csv
    folder = "../Residual, Output, Simulated data/"
    filename = "HBV_output_"+str(file_nr)+".txt"
    filepath = folder + filename
    
    df = pd.read_csv(filepath)
    
    # Rename columns to match LSTM notebook
    df = df.rename(columns={"DATE": "timestamp"})
    
    # Cast timestamp to datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    
    # Remove first column
    df = df.drop(['Unnamed: 0'], axis=1)
    
    return df

In [26]:
read_output_simulated_residual_data(1).head()

,timestamp,OBSRUNOFF,SIMRUNOFF,RESIDUAL
0,2000-01-01,0.597,0.013,0.584
1,2000-02-01,0.597,0.055,0.542
2,2000-03-01,0.597,0.103,0.494
3,2000-04-01,0.641,0.177,0.464
4,2000-05-01,0.685,0.222,0.463
